<a href="https://colab.research.google.com/github/balaorcl/mlbasics/blob/main/s7c2_ml_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Project

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Cleaning

### 1. Introduction

In this course, we will walk through the full data science life cycle, from data cleaning and feature selection to machine learning. we will focus on credit modelling, a well known data science problem that focusses on modelling a borrowers's [credit risk](https://en.wikipedia.org/wiki/Credit_risk).

Credit has played a key role in the economy for centuries and some form of credit has existed since the begining of commerce.

We'll be working financial lending data from [Lending Club](https://www.lendingclub.com/). 

#### Lending Club
Lending Club is a marketplace for personals loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return.

#### Borrower
Higher interest rate means that borrower is risky and more likely not to pay back the loan. Lower interest rate means the borrower has good credit history and more likely to pay back the loan. Interest rate range from 6.46% to 17.74% and each borrower is given a grade. If the borrower accepts the interest rate, then the loan is listed on the Lending Club marketplace.
  ![interest-rate](images/interest-notes_grades_rates_apr2020_latest.png)

#### Investor
Investors are primarily interested in receiving a return on their investments. Approved loans are listed on the Lending Club website, where qualified investors can browse recently approved loans, the borrower's credit score, the purpose of the loan, and other loan information from the application. Once they are ready to support a loan request, they select the amount of money they want to fund. Once a loan request amount is fully 
funded, the borrower receives the money they requested minus the [origination fee](https://help.lendingclub.com/hc/en-us/articles/214463677) that the Lending Club charges.

#### Business Process

Each borrower fills out a comprehensive application, providing their past financial history, the reason for the loan, and more. Lending club evaluates each borrower's credit score using past historical data (and their own data science process!) and assign an interest rate to the borrower. The borrower has to pay back with interest rate. You can read more about the interest rate that Lending Club assigns [here](https://www.lendingclub.com/loans/personal-loans/rates-fees?). Lending club also tries to verify the borrower information, but due to regulation reasons it cannot verify all the borrower information.

The borrower makes monthly payments back to Lending Club either over 36 months or over 60 months. Lending Club redistributes these payments to the investors. This means that investors don't have to wait until the full amount is paid off before they see return in money. If a loan is fully paid off on time, the investors make a return with respect to the interest rate. Many loans are not completely paid off on time, and some borrowers default the loan.

Following is the diagram from [Bible Money Matters](https://www.biblemoneymatters.com/introduction-to-peer-to-peer-lending-signing-up-to-use-lending-club/)
![how-social-lending-works](images/how-social-lending-works.jpg)

#### Assumptions

Lending Club should have effective credit modelling and the investors also should be very careful to determine which loans are likely to be paid off. We may assume that investors will put money into low interest loans, but the returns from high interest loans is very good but with a risk. Most investors use a strategy to invest small amounts in many loans, with healthy mixes of low, medium, and interest loans. We will understand the dataset features, build a machine learning model which will predict if a loan will be paid or not.


### 2. Introduction to Data

Data dictionary is in excel format, **LoanStats** sheet describes the approved loan datasets and the **RejectStats** describes the rejected loans datasets. We will be focussing on approved loans only.
The approved loan datasets contain information on current loans, completed loans, and defaulted loans.

#### Problem Statement

* Can we build a machine learning model that can accurately predict if a borrower will pay their loan on time or not?

In [ ]:
import pandas as pd

### 3. Reading the CSV

We will use the dataset of approved loans from 2007 to 2011.

In [ ]:
loans_2007 = pd.read_csv("drive/My Drive/data/loans_2007.csv")
# print the first row
print(loans_2007.head(0))
# check the number of columns
print(len(loans_2007.columns))

Empty DataFrame
Columns: [id, member_id, loan_amnt, funded_amnt, funded_amnt_inv, term, int_rate, installment, grade, sub_grade, emp_title, emp_length, home_ownership, annual_inc, verification_status, issue_d, loan_status, pymnt_plan, purpose, title, zip_code, addr_state, dti, delinq_2yrs, earliest_cr_line, inq_last_6mths, open_acc, pub_rec, revol_bal, revol_util, total_acc, initial_list_status, out_prncp, out_prncp_inv, total_pymnt, total_pymnt_inv, total_rec_prncp, total_rec_int, total_rec_late_fee, recoveries, collection_recovery_fee, last_pymnt_d, last_pymnt_amnt, last_credit_pull_d, collections_12_mths_ex_med, policy_code, application_type, acc_now_delinq, chargeoff_within_12_mths, delinq_amnt, pub_rec_bankruptcies, tax_liens]
Index: []
52


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 4. First group of columns

The Dataframe contains many columns and can be cumbersome to try to explore all at once. We will break the columns into 3 groups of 18 columns and use the data dictionary to become familiar with each column.

We have to pay attention to any features that:
* formatted poorly and needs to be cleaned up
* require more data or a lot of processing to turn into a useful feature
* contain redundant information
* leak information from the future, which will lead the model to overfit

In [ ]:
# First 18 columns of first row
loans_2007.iloc[0,:19]

id                         1077501
member_id               1.2966e+06
loan_amnt                     5000
funded_amnt                   5000
funded_amnt_inv               4975
term                     36 months
int_rate                    10.65%
installment                 162.87
grade                            B
sub_grade                       B2
emp_title                      NaN
emp_length               10+ years
home_ownership                RENT
annual_inc                   24000
verification_status       Verified
issue_d                   Dec-2011
loan_status             Fully Paid
pymnt_plan                       n
purpose                credit_card
Name: 0, dtype: object

In [ ]:
# Displaying first 5 rows
loans_2007.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


Following features needs to be removed:
* **id**: randomly generated field by Lending Club for unique indentification purposes only
* **member_id**: randonly generated field by Lending Club for unique indentification purposes only
* **funded_amnt**: leaks data from the future(after the loan is already started to be funded)
* **funded_amnt_inv**: leaks data from the future(after the loan is already started to be funded)
* **grade**: contains redundant information as the interest rate column(int_rate)
* **sub_grade**: contains redundant information as the interest rate column(int_rate)
* **emp_title**: requires other data and a lot of processing to be potentially be useful
* **issue_id**: leaks data from the future (after the loan is already completely funded)

### 5. First groups of columns

In [ ]:
loans_2007.drop(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"], axis=1, inplace=True)

In [ ]:
loans_2007.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,3000.0,60 months,12.69%,67.79,1 year,RENT,80000.0,Source Verified,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


### 6. Second group of features

In [ ]:
# Next 18 columns
loans_2007.loc[0, "title":"total_rec_prncp"]

title                  Computer
zip_code                  860xx
addr_state                   AZ
dti                       27.65
delinq_2yrs                   0
earliest_cr_line       Jan-1985
inq_last_6mths                1
open_acc                      3
pub_rec                       0
revol_bal                 13648
revol_util                83.7%
total_acc                     9
initial_list_status           f
out_prncp                     0
out_prncp_inv                 0
total_pymnt             5863.16
total_pymnt_inv         5833.84
total_rec_prncp            5000
Name: 0, dtype: object

Drop the following columns
* zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
* out_prncp: outstanding principal amount for a loan, leaks data from the future, (after the loan already started to be paid off)
* out_prncp_inv: outstanding principal amount for a loan, also leaks data from the future, (after the loan already started to be paid off)
* total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
* total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
* total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)

The out_prncp, out_prncp_inv, total_pymnt columns describe properties of loan after it is fully funded and started to be paid off. This information is not required in our model.

### 7.Second group of features


In [ ]:
# drop columns
loans_2007.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"],
                axis = 1,
                inplace = True)

In [ ]:
loans_2007.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,Fully Paid,n,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,Charged Off,n,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,Fully Paid,n,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,Fully Paid,n,other,personel,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,3000.0,60 months,12.69%,67.79,1 year,RENT,80000.0,Source Verified,Current,n,other,Personal,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


### 8. Third group of features

In [ ]:
# Last 18 columns
loans_2007.loc[0, "total_rec_int":"tax_liens"]

total_rec_int                     863.16
total_rec_late_fee                     0
recoveries                             0
collection_recovery_fee                0
last_pymnt_d                    Jan-2015
last_pymnt_amnt                   171.62
last_credit_pull_d              Jun-2016
collections_12_mths_ex_med             0
policy_code                            1
application_type              INDIVIDUAL
acc_now_delinq                         0
chargeoff_within_12_mths               0
delinq_amnt                            0
pub_rec_bankruptcies                   0
tax_liens                              0
Name: 0, dtype: object

Following columns leaks data from future:
* total_rec_int: leaks data from the future, (after the loan already started to be paid off),
* total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
* recoveries: also leaks data from the future, (after the loan already started to be paid off),
* collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
* last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
* last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).

All of the above mentioned columns leak data from the future, meaning that they describe aspects of loan after it's already been fully funded and started to be paid off by the borrower.



### 9. Third group of features

In [ ]:
# drop columns
loans_2007.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"],
                axis = 1,
                inplace = True)


In [ ]:
loans_2007.head(1)

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,Fully Paid,n,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(len(loans_2007.columns))

32


### 10. Target Column

We reduced the number of columns from 52 to 32, we need to decide on a target column that we want to use for modelling.

We should use the **loan_status** column, since it is the only column describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower.

In [ ]:
loans_2007["loan_status"].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

In [ ]:
print(loans_2007["loan_status"].value_counts())

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64


We need numerical values to train the machine learning model, **loan_status** columns text_values needs to be converted to numerical.

### 11. Binary Classification

Following is detailed explanation of each loan_status:
* Fully Paid - Loan has been fully paid off
* Charged Off - Loans for which no payments is expected
* Does not meet the credit policy. Status:Fully Paid
* Does not meet the credit policy. Status:Charged Off
* In Grace Period - Loan request past is due date, but still in grace period of 15 days
* Late(16 - 30 days) - Not paid for past 16 to 30 days, current payment pending
* Late(31 - 120 days) - Not paid for past 31 to 120 days, current payment pending
* Current - Loan is up to date on current payments
* Default - Loan is defaulted and no payment done for more than 121 days

From the investor's perspective, we are interested in trying to predict which loans will be paid off on time and which ones will not be paid on time. Only **Fully Paid** and **Charged Off** loan status describes the final outcome of the loan.

We can treat this problem as **Binary Classification**, prediction will be of 2 values either **Fully Paid(1)** or **Charged Off(0)**.

#### Class Imbalance

We have 33,136 Fully Paid loans and only 5,634 Charged Off loans which leads to **class imbalance** between positive and negative cases. It is a common problem in binary classification and during training, the model ends up having a strong bias towards predicting the class with more observations in the training set and will rarely predict the class with less observations.

### 12. Binary Classification

In [ ]:
loans_2007

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,Fully Paid,n,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,Charged Off,n,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,Fully Paid,n,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,Fully Paid,n,other,personel,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,3000.0,60 months,12.69%,67.79,1 year,RENT,80000.0,Source Verified,Current,n,other,Personal,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42533,2525.0,36 months,9.33%,80.69,< 1 year,RENT,110000.0,Not Verified,Does not meet the credit policy. Status:Fully ...,n,other,Car repair bill,NY,10.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,f,May-2007,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN
42534,6500.0,36 months,8.38%,204.84,< 1 year,NONE,NaN,Not Verified,Does not meet the credit policy. Status:Fully ...,n,other,Buying a car,NY,4.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,f,Aug-2007,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN
42535,5000.0,36 months,7.75%,156.11,10+ years,MORTGAGE,70000.0,Not Verified,Does not meet the credit policy. Status:Fully ...,n,other,Aroundthehouse,CT,8.81,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,f,Feb-2015,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN
42536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Remove all rows that contain values other than "Fully Paid" or "Charged Off"
# Create a boolean filter
loan_status_bool = loans_2007["loan_status"].isin(["Fully Paid", "Charged Off"]) == False
# get the row index
rows_index = loans_2007.loc[loan_status_bool].index
loans_2007.loc[rows_index,:]["loan_status"].value_counts(dropna=False)
# 3768 rows to be removed

Does not meet the credit policy. Status:Fully Paid     1988
Current                                                 961
Does not meet the credit policy. Status:Charged Off     761
Late (31-120 days)                                       24
In Grace Period                                          20
Late (16-30 days)                                         8
Default                                                   3
NaN                                                       3
Name: loan_status, dtype: int64

In [ ]:
# Check the full loan status count
loans_2007["loan_status"].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

In [ ]:
# Apply the rows index to drop the rows
loans_2007.drop(rows_index, inplace=True)
loans_2007.head()
len(loans_2007)

38770

In [ ]:
loans_2007

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,Fully Paid,n,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,Charged Off,n,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,Fully Paid,n,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,Fully Paid,n,other,personel,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
5,5000.0,36 months,7.90%,156.46,3 years,RENT,36000.0,Source Verified,Fully Paid,n,wedding,My wedding loan I promise to pay back,AZ,11.20,0.0,Nov-2004,3.0,9.0,0.0,7963.0,28.3%,12.0,f,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39781,2500.0,36 months,8.07%,78.42,4 years,MORTGAGE,110000.0,Not Verified,Fully Paid,n,home_improvement,Home Improvement,CO,11.33,0.0,Nov-1990,0.0,13.0,0.0,7274.0,13.1%,40.0,f,Jun-2010,NaN,1.0,INDIVIDUAL,0.0,NaN,0.0,NaN,NaN
39782,8500.0,36 months,10.28%,275.38,3 years,RENT,18000.0,Not Verified,Fully Paid,n,credit_card,Retiring credit card debt,NC,6.40,1.0,Dec-1986,1.0,6.0,0.0,8847.0,26.9%,9.0,f,Jul-2010,NaN,1.0,INDIVIDUAL,0.0,NaN,0.0,NaN,NaN
39783,5000.0,36 months,8.07%,156.84,< 1 year,MORTGAGE,100000.0,Not Verified,Fully Paid,n,debt_consolidation,MBA Loan Consolidation,MA,2.30,0.0,Oct-1998,0.0,11.0,0.0,9698.0,19.4%,20.0,f,Jun-2007,NaN,1.0,INDIVIDUAL,0.0,NaN,0.0,NaN,NaN
39784,5000.0,36 months,7.43%,155.38,< 1 year,MORTGAGE,200000.0,Not Verified,Fully Paid,n,other,JAL Loan,MD,3.72,0.0,Nov-1988,0.0,17.0,0.0,85607.0,0.7%,26.0,f,Jun-2007,NaN,1.0,INDIVIDUAL,0.0,NaN,0.0,NaN,NaN


In [ ]:
# Check the value counts based on loan_status
loans_2007["loan_status"].value_counts()

Fully Paid     33136
Charged Off     5634
Name: loan_status, dtype: int64

In [ ]:
# Replace Fully Paid as 1 and Charged Off as 0 using a dictionary
loans_2007["loan_status"].replace({"Fully Paid":1, "Charged Off":0}, inplace=True)
loans_2007.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,n,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,0,n,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,1,n,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,1,n,other,personel,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
5,5000.0,36 months,7.90%,156.46,3 years,RENT,36000.0,Source Verified,1,n,wedding,My wedding loan I promise to pay back,AZ,11.20,0.0,Nov-2004,3.0,9.0,0.0,7963.0,28.3%,12.0,f,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [ ]:
loans_2007

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,n,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,0,n,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,1,n,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,1,n,other,personel,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
5,5000.0,36 months,7.90%,156.46,3 years,RENT,36000.0,Source Verified,1,n,wedding,My wedding loan I promise to pay back,AZ,11.20,0.0,Nov-2004,3.0,9.0,0.0,7963.0,28.3%,12.0,f,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39781,2500.0,36 months,8.07%,78.42,4 years,MORTGAGE,110000.0,Not Verified,1,n,home_improvement,Home Improvement,CO,11.33,0.0,Nov-1990,0.0,13.0,0.0,7274.0,13.1%,40.0,f,Jun-2010,NaN,1.0,INDIVIDUAL,0.0,NaN,0.0,NaN,NaN
39782,8500.0,36 months,10.28%,275.38,3 years,RENT,18000.0,Not Verified,1,n,credit_card,Retiring credit card debt,NC,6.40,1.0,Dec-1986,1.0,6.0,0.0,8847.0,26.9%,9.0,f,Jul-2010,NaN,1.0,INDIVIDUAL,0.0,NaN,0.0,NaN,NaN
39783,5000.0,36 months,8.07%,156.84,< 1 year,MORTGAGE,100000.0,Not Verified,1,n,debt_consolidation,MBA Loan Consolidation,MA,2.30,0.0,Oct-1998,0.0,11.0,0.0,9698.0,19.4%,20.0,f,Jun-2007,NaN,1.0,INDIVIDUAL,0.0,NaN,0.0,NaN,NaN
39784,5000.0,36 months,7.43%,155.38,< 1 year,MORTGAGE,200000.0,Not Verified,1,n,other,JAL Loan,MD,3.72,0.0,Nov-1988,0.0,17.0,0.0,85607.0,0.7%,26.0,f,Jun-2007,NaN,1.0,INDIVIDUAL,0.0,NaN,0.0,NaN,NaN


### 13. Removing single value columns

Columns with Unique values will not be useful for the model, as they do not add any information to each loan application. We need to count number of unique values in each column. We have to drop NaN and then check for unique for each column.

In [ ]:
# Lets check for the Unique value columns, we have to iterate through each columns in the dataframe
drop_columns = []
for (columnName, ColumnData) in loans_2007.iteritems():
    ColumnData.dropna(inplace=True)
    if len(ColumnData.unique()) == 1:
        drop_columns.append(columnName)
drop_columns

['pymnt_plan',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'tax_liens']

In [ ]:
loans_2007

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,n,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,0,n,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,1,n,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,1,n,other,personel,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
5,5000.0,36 months,7.90%,156.46,3 years,RENT,36000.0,Source Verified,1,n,wedding,My wedding loan I promise to pay back,AZ,11.20,0.0,Nov-2004,3.0,9.0,0.0,7963.0,28.3%,12.0,f,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39781,2500.0,36 months,8.07%,78.42,4 years,MORTGAGE,110000.0,Not Verified,1,n,home_improvement,Home Improvement,CO,11.33,0.0,Nov-1990,0.0,13.0,0.0,7274.0,13.1%,40.0,f,Jun-2010,NaN,1.0,INDIVIDUAL,0.0,NaN,0.0,NaN,NaN
39782,8500.0,36 months,10.28%,275.38,3 years,RENT,18000.0,Not Verified,1,n,credit_card,Retiring credit card debt,NC,6.40,1.0,Dec-1986,1.0,6.0,0.0,8847.0,26.9%,9.0,f,Jul-2010,NaN,1.0,INDIVIDUAL,0.0,NaN,0.0,NaN,NaN
39783,5000.0,36 months,8.07%,156.84,< 1 year,MORTGAGE,100000.0,Not Verified,1,n,debt_consolidation,MBA Loan Consolidation,MA,2.30,0.0,Oct-1998,0.0,11.0,0.0,9698.0,19.4%,20.0,f,Jun-2007,NaN,1.0,INDIVIDUAL,0.0,NaN,0.0,NaN,NaN
39784,5000.0,36 months,7.43%,155.38,< 1 year,MORTGAGE,200000.0,Not Verified,1,n,other,JAL Loan,MD,3.72,0.0,Nov-1988,0.0,17.0,0.0,85607.0,0.7%,26.0,f,Jun-2007,NaN,1.0,INDIVIDUAL,0.0,NaN,0.0,NaN,NaN


In [ ]:
# Drop the Unique value columns
loans_2007.drop(columns=drop_columns, inplace=True)

In [ ]:
loans_2007.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'last_credit_pull_d',
       'pub_rec_bankruptcies'],
      dtype='object')

In [ ]:
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


In [ ]:
# Export the cleaned dataframe to csv
loans_2007.to_csv("drive/My Drive/data/filtered_loans_2007.csv")

## Preparing the Features

### 1. Check missing values

We have removed the the following from the DataFrame:

* Columns with redundant information
* Columns which are not useful for modelling
* Data which require too much processing to make useful
* Future information

Now we will focus on **Preparing the Features** like
* Handling missing values
* Converting categorical columns to numeric columns
* Removing any other unwanted columns

Machine Learning Model assumes the data is numerical and contains no mising values.\
**scikit-learn** will return error, if the model is trained on data with missing values or non-numeric values.

In [ ]:
loans = pd.read_csv("drive/My Drive/data/filtered_loans_2007.csv")

In [ ]:
null_counts = loans.isnull().sum()
print(null_counts)

Unnamed: 0                 0
loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


### 2. Handling missing values

Following columns have 50 or less rows with missing values
* title - 11
* revol_util - 50
* last_credit_pull_d - 2

Following columns have 697 or more rows with missing values
* emp_length - 1036
* pub_rec_bankruptcies - 697

As per the functional domain knowledge, employment length is mostly used in assessing how risky a potential borrower is, we will keep this column despite more missing values.

We will focus on the values of *pub_rec_bankruptcies* column.

In [ ]:
# check the value count of pub_rec_bankruptcies
print(loans.pub_rec_bankruptcies.value_counts())

0.0    36422
1.0     1646
2.0        5
Name: pub_rec_bankruptcies, dtype: int64


In [ ]:
# check the value count of pub_rec_bankruptcies with normalization and NaN 
print(loans.pub_rec_bankruptcies.value_counts(normalize=True, dropna=False))

0.0    0.939438
1.0    0.042456
NaN    0.017978
2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64


As 94% of *pub_rec_bankruptcies* values are in the same category, which will not be helpful in model prediction.
*pub_rec_bankruptcies* column to be dropped, we will keep the following columns and remove only the missing rows.
* emp_length
* title
* revol_util

* last_credit_pull_d

In [ ]:
# Drop the pub_rec_bankruptcies column
loans.drop("pub_rec_bankruptcies", axis=1, inplace=True)

In [ ]:
# Remove all rows with null values from the DataFrame
loans.dropna(inplace=True)

In [ ]:
# Check the value_counts of column datatypes
print(loans.dtypes.value_counts(dropna=False))

object     11
float64    10
int64       2
dtype: int64


### 3. Text Columns

Text columns needs to be converted to numerical data types.

In [ ]:
float_df = loans.select_dtypes(include="float")
float_df

,loan_amnt,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc
0,5000.0,162.87,24000.0,27.65,0.0,1.0,3.0,0.0,13648.0,9.0
1,2500.0,59.83,30000.0,1.00,0.0,5.0,3.0,0.0,1687.0,4.0
2,2400.0,84.33,12252.0,8.72,0.0,2.0,2.0,0.0,2956.0,10.0
3,10000.0,339.31,49200.0,20.00,0.0,1.0,10.0,0.0,5598.0,37.0
4,5000.0,156.46,36000.0,11.20,0.0,3.0,9.0,0.0,7963.0,12.0
...,...,...,...,...,...,...,...,...,...,...
38765,2500.0,78.42,110000.0,11.33,0.0,0.0,13.0,0.0,7274.0,40.0
38766,8500.0,275.38,18000.0,6.40,1.0,1.0,6.0,0.0,8847.0,9.0
38767,5000.0,156.84,100000.0,2.30,0.0,0.0,11.0,0.0,9698.0,20.0
38768,5000.0,155.38,200000.0,3.72,0.0,0.0,17.0,0.0,85607.0,26.0


In [ ]:
# Lets make a dataframe of "object" datatype
object_columns_df = loans.select_dtypes(include="object")
print(object_columns_df.head(1))

         term int_rate  ... revol_util last_credit_pull_d
0   36 months   10.65%  ...      83.7%           Jun-2016

[1 rows x 11 columns]


### 4. Converting text columns

Following columns represent categorical values:

* home_ownership - it can be 1 out of 4 values accroding to data dictionary
* verification_status - indicates if verified by Lending Club
* emp_length - number of years the borrower was employed upon the time of application
* term - number of payments on the loan, either 36 or 60
* addr_state - borrower's state of residence
* purpose - category provided by the borrower for the loan request
* title - loan title provided by the borrower

Following columns represent numeric values:
* int_rate - interest rate of the loan in %
* revol_util - revolving credit is amount of credit the borrower is using relative to available credit like credit card limit

Following columns contain date values which requires Feature engineering to use it effectively:
* earliest_cr_line - the month the borrower's earliest reported credit line was opened
* last_credit_pull_d - the most recent month Lending Club pulled credit for this loan

### 5. First 5 categorical columns

In [ ]:
# Let's explore the first 5 categorical columns
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for columnName in cols:
    print(loans[columnName].value_counts())

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
 36 months    28234
 60 months     9441
Name: term, dtype: int64
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
AL     420
LA     420
KY     311
OK     285
KS     249
UT     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162
NH     157
DE     110
MT      77
WY      76
AK      76
SD      60
VT  

### 6. The reason for the loan

The **home_ownership**, **verification_status**, **emp_length**, **term**, and **addr_state** columns all contain multiple discrete values. **emp_length** column should be cleaned and treated as numeric one as it contains number of years.\
We have to decide between **purpose** and **title** columns as both columns represent similar meaning.

In [ ]:
loans.purpose.value_counts()

debt_consolidation    17751
credit_card            4911
other                  3711
home_improvement       2808
major_purchase         2083
small_business         1719
car                    1459
wedding                 916
medical                 655
moving                  552
house                   356
vacation                348
educational             312
renewable_energy         94
Name: purpose, dtype: int64

In [ ]:
loans.title.value_counts()

Debt Consolidation                2068
Debt Consolidation Loan           1599
Personal Loan                      624
Consolidation                      488
debt consolidation                 466
                                  ... 
Lambright1                           1
Mom's car                            1
I'm Eliminating My Debt!             1
consolidating credit card debt       1
unexpected bills                     1
Name: title, Length: 18881, dtype: int64

### 7. Categorical columns

The **home_ownership**, **verification_status**, **emp_length**, and **term** columns each contain a few discrete categorical values, which will be encoded as dummy variables.\

**purpose** and **title** contains overlapping information, we will keep **purpose** column as it contains few discrete values. **title** column has data quality issues as many values are repeated like **Debt Consolidation** and **Debt Cosolidation Loan**.\

We will use the following mapping to clean **emp_length** column:
* < 1 year - 0
* 1 year - 1
* 2 years - 2
* 3 years - 3
* 4 years - 4
* 5 years - 5
* 6 years - 6
* 7 years - 7
* 8 years - 8
* 9 years - 9
* 10+ years - 10

In [ ]:
loans.addr_state.describe()

count     37675
unique       50
top          CA
freq       6776
Name: addr_state, dtype: object

**addr_state** contains 50 different unique values, encoding it will result in 50 columns and DataFrame will become larger.
It is better to remove this column from consideration.

In [ ]:
# Remove the last_credit_pull_d, addr_state, title and earliest_cr_line from loans
loans.drop(columns=["last_credit_pull_d", "addr_state", "title", "earliest_cr_line"], inplace=True)

In [ ]:
loans.int_rate

0         10.65%
1         15.27%
2         15.96%
3         13.49%
4          7.90%
          ...   
38765      8.07%
38766     10.28%
38767      8.07%
38768      7.43%
38769     13.75%
Name: int_rate, Length: 37675, dtype: object

In [ ]:
loans.revol_util

0        83.7%
1         9.4%
2        98.5%
3          21%
4        28.3%
         ...  
38765    13.1%
38766    26.9%
38767    19.4%
38768     0.7%
38769    51.5%
Name: revol_util, Length: 37675, dtype: object

In [ ]:
# Convert the int_rate column to float column
loans.int_rate = loans["int_rate"].str.rstrip('%').astype("float")

In [ ]:
loans.int_rate

0        10.65
1        15.27
2        15.96
3        13.49
4         7.90
         ...  
38765     8.07
38766    10.28
38767     8.07
38768     7.43
38769    13.75
Name: int_rate, Length: 37675, dtype: float64

In [ ]:
# Convert the revol_util column to float column
loans.revol_util = loans["revol_util"].str.rstrip('%').astype("float")
loans.revol_util

0        83.7
1         9.4
2        98.5
3        21.0
4        28.3
         ... 
38765    13.1
38766    26.9
38767    19.4
38768     0.7
38769    51.5
Name: revol_util, Length: 37675, dtype: float64

In [ ]:
# emp_length columns to be replaced with numerical values
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans.replace(mapping_dict, inplace=True)

In [ ]:
loans.emp_length

0        10
1         0
2        10
3        10
4         3
         ..
38765     4
38766     3
38767     0
38768     0
38769     0
Name: emp_length, Length: 37675, dtype: int64

### 8. Dummy Variables

Let's encode **home_ownership**, **verification_status**, **purpose**, **term** columns as dummy variables to use in the model.

In [ ]:
# returns a dataframe of 1 column for each dummy variable
loans.term

0         36 months
1         60 months
2         36 months
3         36 months
4         36 months
            ...    
38765     36 months
38766     36 months
38767     36 months
38768     36 months
38769     36 months
Name: term, Length: 37675, dtype: object

In [ ]:
dummy_df = pd.get_dummies(loans.term)
dummy_df

,36 months,60 months
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0
...,...,...
38765,1,0
38766,1,0
38767,1,0
38768,1,0


In [ ]:
dummy_df = pd.get_dummies(loans[["term", "verification_status"]])
dummy_df

,term_ 36 months,term_ 60 months,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,1,0,0,0,1
1,0,1,0,1,0
2,1,0,1,0,0
3,1,0,0,1,0
4,1,0,0,1,0
...,...,...,...,...,...
38765,1,0,1,0,0
38766,1,0,1,0,0
38767,1,0,1,0,0
38768,1,0,1,0,0


In [ ]:
# Encode home_ownership, verification_status, purpose and term columns as integer values
dummy_df = pd.get_dummies(loans[["home_ownership", "verification_status", "purpose", "term"]])
dummy_df

,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38765,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
38766,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
38767,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
38768,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [ ]:
# Add the dummy_columns to loans dataframe
loans = pd.concat([loans, dummy_df], axis=1)

In [ ]:
# drop the orginal columns
loans.drop(columns=["home_ownership", "verification_status", "purpose", "term"], inplace=True)

In [ ]:
loans.head()

,Unnamed: 0,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,2400.0,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,3,10000.0,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,5,5000.0,7.90,156.46,3,36000.0,1,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [ ]:
# Drop Unnamed column
loans.drop(columns = ["Unnamed: 0"], inplace=True)

In [ ]:
loans.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,5000.0,7.90,156.46,3,36000.0,1,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


## Making Predictions

Features need to be generated which will be the inputs for Machine Learning algorithm. The algorithm will make predictions based on **loan_status** column.

We need generate features from the data, which can be fed into a machine learning algorithm.

In [ ]:
loans["loan_status"].value_counts()

1    32286
0     5389
Name: loan_status, dtype: int64

Class imbalance exists with respect to the loan_status column, where by positive cases are 6 times more than the negative cases.

In [ ]:
print(loans.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37675 entries, 0 to 38769
Data columns (total 38 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            37675 non-null  float64
 1   int_rate                             37675 non-null  float64
 2   installment                          37675 non-null  float64
 3   emp_length                           37675 non-null  int64  
 4   annual_inc                           37675 non-null  float64
 5   loan_status                          37675 non-null  int64  
 6   dti                                  37675 non-null  float64
 7   delinq_2yrs                          37675 non-null  float64
 8   inq_last_6mths                       37675 non-null  float64
 9   open_acc                             37675 non-null  float64
 10  pub_rec                              37675 non-null  float64
 11  revol_bal                   

### 2. Picking an error metric

An error metric will help us figure out our model's performance good or poor.
We are primarily concerned with False Positives and False Negatives.

#### False Positives

Model predicts that a loan will be paid on time, but it is not.

#### False Negatives

Model predicts that a loan will not be paid on time, but it is paid on time.

| Actual Status | Prediction | Error Type |
| -- | -- | -- |
| 0 | 1 | False Positive |
| 1 | 1 | True Positive |
| 0 | 0 | True Negative |
| 1 | 0 | False Negative|

A conservative investor would want to minimize risk, and avoid False Positives as much as possible. They will be okay with missed oppurtunities(False Negatives) instead of investing in a risky loan.


### 3. Picking an error metric

In [ ]:
import numpy as np

predictions = np.array(loans["loan_status"])
print(predictions)

[1 0 1 ... 1 1 1]


In [ ]:
# Assuming predictions by shuffling
np.random.shuffle(predictions)
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
# Finding number of True Negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
tn

790

In [ ]:
# Finding number of True Positives
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
tp

27687

In [ ]:
# Finding number of False Negatives
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
fn

4599

In [ ]:
# Finding number of False Positives
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
fp

4599

### 4. Class imbalance

  There are 6 times as many loans that are paid off time (1), than loans that were not paid off on time (0). This causes a major issue when we use accuracy as a metric. Assume we invest 1000 each, on 7 loans. Each borrower pays us 10% interest back, we make profit of 100 dollars on each loan.

  | Actual | Prediction | Profit/Loss |
  | -- | -- | -- |
  | 0 | 1 | -1000 |
  | 1 | 1 | 100 |
  | 1 | 1 | 100 |
  | 1 | 1 | 100 |
  | 1 | 1 | 100 |
  | 1 | 1 | 100 |
  | 1 | 1 | 100 |

We made 600 dollars profit, but we end up losing 400 dollars, even though our model is technically correct. In this scenario, we should use metrics that tells the number of false positives and false negatives.

We should optimize for:
* high recall
* low fall-out

FPR = FP / (FP + TN)

TPR = TP / (TP + FN)

* False Positive Rate = Percentage of loans that should not be funded that I may fund based on model's recommendations
* True Positive Rate = Percentage of loans that should be funded that I must fund based on model's recommendations




### 5. FPR and TPR

In [ ]:
# Predict that all loans will be paid off on time.
predictions = pd.Series(np.ones(loans.shape[0]))
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
tn = 0
fpr = fp / (fp + tn)
fpr

1.0

In [ ]:
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
fn = 0
tpr = tp /(tp + fn)
tpr

1.0

### 6. Logistic Regression

We noticed FPR and TPR were 1, because we predicted True Positive(1) for all loans. It means we correctly identified all of the good loans, but failed to identify the all of the bad loans. Now we have setup error metrics, we can make predictions using a machine learning algorithm.

**Logistic regression** algorithm is best for binary classification problems.



In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
# Create features dataframe and remove loan_status column
features = pd.DataFrame(loans)
features.drop(['loan_status'], axis = 1, inplace = True)
features

,loan_amnt,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10,12252.0,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10,49200.0,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,5000.0,7.90,156.46,3,36000.0,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38765,2500.0,8.07,78.42,4,110000.0,11.33,0.0,0.0,13.0,0.0,7274.0,13.1,40.0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
38766,8500.0,10.28,275.38,3,18000.0,6.40,1.0,1.0,6.0,0.0,8847.0,26.9,9.0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
38767,5000.0,8.07,156.84,0,100000.0,2.30,0.0,0.0,11.0,0.0,9698.0,19.4,20.0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
38768,5000.0,7.43,155.38,0,200000.0,3.72,0.0,0.0,17.0,0.0,85607.0,0.7,26.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [ ]:
# Create a Series named target with only loan_status column
target = pd.Series(loans["loan_status"])
target

0        1
1        0
2        1
3        1
4        1
        ..
38765    1
38766    1
38767    1
38768    1
38769    1
Name: loan_status, Length: 37675, dtype: int64

In [ ]:
# Fit the logistic regression model, Train the model
X = features
y = target
lr.fit(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
predictions = lr.predict(X)
predictions

array([1, 1, 1, ..., 1, 1, 1])

### 7. Cross Validation

The predictions are overfit, as the same data is used for training and prediction as well. We will do k-fold cross validation to get the real accuracy of the model.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

# set the model
lr = LogisticRegression()
# Make predictions using 3-fold cross-validation
predictions = cross_val_predict(lr, X, y, cv=3)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [ ]:
predictions = pd.Series(predictions)
predictions

0        1
1        1
2        1
3        1
4        1
        ..
37670    1
37671    1
37672    1
37673    1
37674    1
Length: 37675, dtype: int64

In [ ]:
# calculate tpr
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
tpr = tp / (tp + fn)
tpr

0.9984382470119522

In [ ]:
# Calculate FPR
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
fpr = fp / (fp + tn)
fpr

0.9978886756238003

In [ ]:
predictions.value_counts()

1    37613
0       62
dtype: int64

### 8. Penalizing the classifier
Eventhough we are not using accuracy as an error metric, the classifier is not accounting for the imbalance in classes. There are few ways to get a classifier to correct for imbalances classes. Following are the 2 ways:
* Use oversampling and undersampling to ensure the classifier gets input that has a balanced number of each class
* Tell the classifier to penalize misclassifications of the less number of class - 0 than the more number of class - 1

Oversampling and undersampling involves taking a sample of equal number of rows where *loan_status* is *1* and *loan_status* is *0*. By this method the prediction will be of only *50%* accuracy.

We can use scikit-learn to penalize the misclassification of the minority class during the training process. We can do this by setting the **class_weight** parameter to **balanced** when the creating LogisticRegression instance. The penalty means that the LogisticRegression classifier pays more attention to correctly classifying rows where *loan_status* is *0*.

By setting the **class_weight** parameter to **balanced**, the penalty is set inversely proportional to the class frequencies i.e. for the classifier, correctly classifying a row where *loan_status* is *0* is *6* times more important than correctly classifying a row where *loan_status* is *1*.


### 9. Penalizing the classifier

In [ ]:
lr = LogisticRegression(class_weight="balanced")
predictions = cross_val_predict(lr, X, y)
predictions = pd.Series(predictions)
predictions

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0        0
1        0
2        0
3        1
4        1
        ..
37670    1
37671    0
37672    1
37673    1
37674    0
Length: 37675, dtype: int64

In [ ]:
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
tpr = tp / (tp + fn)
tpr

0.5206693227091633

In [ ]:
# Calculate FPR
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
fpr = fp / (fp + tn)
fpr

0.5080614203454894

### 10. Manual penalities

FPR and TPR is slightly improved, a conservative investor would like to have a low FPR to avoid bad loans. We can also specify penalty manually to overcome class imbalance.

In [ ]:
# penalty of 10 for misclassifying a 0
# penalty of 1 for misclassifying a 1
penalty = {0: 10,
           1: 1}
lr = LogisticRegression(class_weight=penalty, max_iter=1000)
predictions = cross_val_predict(lr, X, y, cv=3)
predictions = pd.Series(predictions)
predictions           

0        0
1        0
2        0
3        0
4        0
        ..
37670    1
37671    0
37672    1
37673    1
37674    0
Length: 37675, dtype: int64

In [ ]:
# Calculate FPR
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
fpr = fp / (fp + tn)
fpr

0.1489443378119002

In [ ]:
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
tpr = tp / (tp + fn)
tpr

0.15439043824701196

### 11. Random Forests

Assigning manual penalties lowered the FPR, thus lowered the risk of investing in bad loans. We can try Random Forest, which can work woth nonlinear data. Logistic Regression is best for linear data. Training a Random Forest may give better accuracy, as the columns correlate nonlinearly with *loan_status*.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
# random_state = 1, so predictions don't vary due to random chance
rf = RandomForestClassifier(class_weight="balanced", random_state=1)
predictions = cross_val_predict(rf, X, y, cv=3)
predictions = pd.Series(predictions)
predictions 

0        1
1        1
2        1
3        1
4        1
        ..
37670    1
37671    1
37672    1
37673    1
37674    1
Length: 37675, dtype: int64

In [ ]:
# Calculate FPR
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
fpr = fp / (fp + tn)
fpr

0.9973128598848369

In [ ]:
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
tpr = tp / (tp + fn)
tpr

0.9967171314741036

### 12. Conclusion

Random Forest Classifier did not improve our FPR. The model is predicting mostly 1. Following can be done to improve the model further:
* Tweak the penalities further
* Try models other than random forest and logistic regression
* Use some of the columns which are dropped earlier
* Ensemble multiple models to get more accurate predictions
* Tune the parameters of the algorithm for higher performance
